In [51]:
# Create your first MLP in Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential,load_model
import numpy
import pandas as pd
#Importing necessary packages
import tensorflow as tf
from sklearn.model_selection import GridSearchCV, KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [11]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
forest = pd.read_csv("forestfires.csv")


In [12]:
forest

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small


In [13]:
forest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   month          517 non-null    object 
 1   day            517 non-null    object 
 2   FFMC           517 non-null    float64
 3   DMC            517 non-null    float64
 4   DC             517 non-null    float64
 5   ISI            517 non-null    float64
 6   temp           517 non-null    float64
 7   RH             517 non-null    int64  
 8   wind           517 non-null    float64
 9   rain           517 non-null    float64
 10  area           517 non-null    float64
 11  dayfri         517 non-null    int64  
 12  daymon         517 non-null    int64  
 13  daysat         517 non-null    int64  
 14  daysun         517 non-null    int64  
 15  daythu         517 non-null    int64  
 16  daytue         517 non-null    int64  
 17  daywed         517 non-null    int64  
 18  monthapr  

In [14]:
forest.isnull().sum()

month            0
day              0
FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
area             0
dayfri           0
daymon           0
daysat           0
daysun           0
daythu           0
daytue           0
daywed           0
monthapr         0
monthaug         0
monthdec         0
monthfeb         0
monthjan         0
monthjul         0
monthjun         0
monthmar         0
monthmay         0
monthnov         0
monthoct         0
monthsep         0
size_category    0
dtype: int64

In [15]:
forest.drop(["month","day"],axis=1,inplace=True)

In [16]:
forest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 29 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   FFMC           517 non-null    float64
 1   DMC            517 non-null    float64
 2   DC             517 non-null    float64
 3   ISI            517 non-null    float64
 4   temp           517 non-null    float64
 5   RH             517 non-null    int64  
 6   wind           517 non-null    float64
 7   rain           517 non-null    float64
 8   area           517 non-null    float64
 9   dayfri         517 non-null    int64  
 10  daymon         517 non-null    int64  
 11  daysat         517 non-null    int64  
 12  daysun         517 non-null    int64  
 13  daythu         517 non-null    int64  
 14  daytue         517 non-null    int64  
 15  daywed         517 non-null    int64  
 16  monthapr       517 non-null    int64  
 17  monthaug       517 non-null    int64  
 18  monthdec  

In [17]:
forest["size_category"].value_counts()

small    378
large    139
Name: size_category, dtype: int64

In [18]:
forest.loc[forest["size_category"]=='small','size_category']=0
forest.loc[forest["size_category"]=='large','size_category']=1

In [19]:
forest["size_category"].value_counts()

0    378
1    139
Name: size_category, dtype: int64

In [20]:
forest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 29 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   FFMC           517 non-null    float64
 1   DMC            517 non-null    float64
 2   DC             517 non-null    float64
 3   ISI            517 non-null    float64
 4   temp           517 non-null    float64
 5   RH             517 non-null    int64  
 6   wind           517 non-null    float64
 7   rain           517 non-null    float64
 8   area           517 non-null    float64
 9   dayfri         517 non-null    int64  
 10  daymon         517 non-null    int64  
 11  daysat         517 non-null    int64  
 12  daysun         517 non-null    int64  
 13  daythu         517 non-null    int64  
 14  daytue         517 non-null    int64  
 15  daywed         517 non-null    int64  
 16  monthapr       517 non-null    int64  
 17  monthaug       517 non-null    int64  
 18  monthdec  

In [35]:
forest["size_category"] = pd.to_numeric(forest.size_category, errors='coerce')

In [36]:
forest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 29 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   FFMC           517 non-null    float64
 1   DMC            517 non-null    float64
 2   DC             517 non-null    float64
 3   ISI            517 non-null    float64
 4   temp           517 non-null    float64
 5   RH             517 non-null    int64  
 6   wind           517 non-null    float64
 7   rain           517 non-null    float64
 8   area           517 non-null    float64
 9   dayfri         517 non-null    int64  
 10  daymon         517 non-null    int64  
 11  daysat         517 non-null    int64  
 12  daysun         517 non-null    int64  
 13  daythu         517 non-null    int64  
 14  daytue         517 non-null    int64  
 15  daywed         517 non-null    int64  
 16  monthapr       517 non-null    int64  
 17  monthaug       517 non-null    int64  
 18  monthdec  

In [37]:
X = forest.iloc[:,:-1]
y = forest.iloc[:,-1:]

In [38]:
X

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1,...,0,0,0,0,0,1,0,0,0,0
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0,...,0,0,0,0,0,0,0,0,1,0
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0,...,0,0,0,0,0,0,0,0,1,0
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1,...,0,0,0,0,0,1,0,0,0,0
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0,...,0,0,0,0,0,0,0,0,0,0
513,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0,...,0,0,0,0,0,0,0,0,0,0
514,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0,...,0,0,0,0,0,0,0,0,0,0
515,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
y

,size_category
0,0
1,0
2,0
3,0
4,0
...,...
512,1
513,1
514,1
515,0


In [40]:
from sklearn.preprocessing import StandardScaler

In [41]:
# Standardization
a = StandardScaler()
a.fit(X)
X_standardized = a.transform(X)

In [42]:
X_standardized

array([[-8.05959472e-01, -1.32332557e+00, -1.83047676e+00, ...,
        -4.40225453e-02, -1.72859706e-01, -7.06081245e-01],
       [-8.10203395e-03, -1.17954077e+00,  4.88890915e-01, ...,
        -4.40225453e-02,  5.78503817e+00, -7.06081245e-01],
       [-8.10203395e-03, -1.04982188e+00,  5.60715454e-01, ...,
        -4.40225453e-02,  5.78503817e+00, -7.06081245e-01],
       ...,
       [-1.64008316e+00, -8.46647711e-01,  4.74768113e-01, ...,
        -4.40225453e-02, -1.72859706e-01, -7.06081245e-01],
       [ 6.80956663e-01,  5.49002541e-01,  2.69382214e-01, ...,
        -4.40225453e-02, -1.72859706e-01, -7.06081245e-01],
       [-2.02087875e+00, -1.68591332e+00, -1.78044169e+00, ...,
         2.27156334e+01, -1.72859706e-01, -7.06081245e-01]])

In [43]:
pd.DataFrame(X_standardized).describe()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
count,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,...,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02
mean,-1.754024e-15,3.070830e-16,7.387171e-17,-3.865380e-17,2.005703e-16,3.362881e-16,-2.676776e-16,-2.841054e-16,-1.274502e-16,4.874674e-17,...,7.179943e-16,-1.933764e-16,-2.260174e-17,1.352883e-17,1.169277e-16,2.265542e-16,-2.596515e-16,1.443075e-16,6.253326e-16,4.024290e-16
std,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,...,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00
min,-1.304582e+01,-1.715608e+00,-2.179108e+00,-1.980578e+00,-2.876943e+00,-1.796637e+00,-2.021098e+00,-7.326831e-02,-2.020198e-01,-4.435755e-01,...,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,-7.060812e-01
25%,-8.063453e-02,-6.606652e-01,-4.448281e-01,-5.535954e-01,-5.842379e-01,-6.924563e-01,-7.361236e-01,-7.326831e-02,-2.020198e-01,-4.435755e-01,...,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,-7.060812e-01
50%,1.732292e-01,-4.020255e-02,4.691190e-01,-1.364774e-01,7.082076e-02,-1.403660e-01,-9.833712e-03,-7.326831e-02,-1.938429e-01,-4.435755e-01,...,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,-7.060812e-01
75%,4.089598e-01,4.927389e-01,6.696628e-01,3.904086e-01,6.741643e-01,5.344111e-01,4.929823e-01,-7.326831e-02,-9.870852e-02,-4.435755e-01,...,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,1.416268e+00
max,1.007353e+00,2.819865e+00,1.261610e+00,1.033538e+01,2.484195e+00,3.417549e+00,3.007063e+00,2.157228e+01,1.695111e+01,2.254407e+00,...,7.512952e+00,4.984977e+00,1.604681e+01,3.893103e+00,5.423261e+00,2.928152e+00,1.604681e+01,2.271563e+01,5.785038e+00,1.416268e+00


In [45]:
# create model1
model = Sequential()
model.add(layers.Dense(10, input_dim=28,activation='relu'))
model.add(layers.Dense(8,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [46]:
# Compile model1
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [47]:
# Fit the model1
model.fit(X_standardized,y, validation_split=0.33, epochs=250, batch_size=10)

Epoch 1/250
35/35 [==============================] - 1s 14ms/step - loss: 0.6402 - accuracy: 0.6213 - val_loss: 0.7038 - val_accuracy: 0.6491
Epoch 2/250
35/35 [==============================] - 0s 3ms/step - loss: 0.5896 - accuracy: 0.7504 - val_loss: 0.6937 - val_accuracy: 0.6784
Epoch 3/250
35/35 [==============================] - 0s 2ms/step - loss: 0.5810 - accuracy: 0.7504 - val_loss: 0.6878 - val_accuracy: 0.6784
Epoch 4/250
35/35 [==============================] - 0s 2ms/step - loss: 0.5381 - accuracy: 0.7699 - val_loss: 0.6846 - val_accuracy: 0.6784
Epoch 5/250
35/35 [==============================] - 0s 2ms/step - loss: 0.5035 - accuracy: 0.7724 - val_loss: 0.6808 - val_accuracy: 0.6784
Epoch 6/250
35/35 [==============================] - 0s 2ms/step - loss: 0.5170 - accuracy: 0.7629 - val_loss: 0.6774 - val_accuracy: 0.6784
Epoch 7/250
35/35 [==============================] - 0s 3ms/step - loss: 0.4904 - accuracy: 0.7798 - val_loss: 0.6711 - val_accuracy: 0.6784
Epoch 8/250


Epoch 59/250
35/35 [==============================] - 0s 3ms/step - loss: 0.1089 - accuracy: 0.9764 - val_loss: 0.8049 - val_accuracy: 0.7778
Epoch 60/250
35/35 [==============================] - 0s 3ms/step - loss: 0.1009 - accuracy: 0.9705 - val_loss: 0.8148 - val_accuracy: 0.7719
Epoch 61/250
35/35 [==============================] - 0s 2ms/step - loss: 0.1143 - accuracy: 0.9664 - val_loss: 0.8335 - val_accuracy: 0.7719
Epoch 62/250
35/35 [==============================] - 0s 2ms/step - loss: 0.1216 - accuracy: 0.9622 - val_loss: 0.8250 - val_accuracy: 0.7602
Epoch 63/250
35/35 [==============================] - 0s 2ms/step - loss: 0.1145 - accuracy: 0.9620 - val_loss: 0.8455 - val_accuracy: 0.7719
Epoch 64/250
35/35 [==============================] - 0s 3ms/step - loss: 0.1223 - accuracy: 0.9449 - val_loss: 0.8554 - val_accuracy: 0.7661
Epoch 65/250
35/35 [==============================] - 0s 3ms/step - loss: 0.1288 - accuracy: 0.9627 - val_loss: 0.8441 - val_accuracy: 0.7719
Epoch 

35/35 [==============================] - 0s 3ms/step - loss: 0.0155 - accuracy: 1.0000 - val_loss: 1.2262 - val_accuracy: 0.7836
Epoch 117/250
35/35 [==============================] - 0s 2ms/step - loss: 0.0165 - accuracy: 1.0000 - val_loss: 1.2335 - val_accuracy: 0.7836
Epoch 118/250
35/35 [==============================] - 0s 2ms/step - loss: 0.0154 - accuracy: 1.0000 - val_loss: 1.2436 - val_accuracy: 0.7836
Epoch 119/250
35/35 [==============================] - 0s 3ms/step - loss: 0.0207 - accuracy: 1.0000 - val_loss: 1.2443 - val_accuracy: 0.7836
Epoch 120/250
35/35 [==============================] - 0s 3ms/step - loss: 0.0136 - accuracy: 1.0000 - val_loss: 1.2517 - val_accuracy: 0.7836
Epoch 121/250
35/35 [==============================] - 0s 3ms/step - loss: 0.0112 - accuracy: 1.0000 - val_loss: 1.2607 - val_accuracy: 0.7953
Epoch 122/250
35/35 [==============================] - 0s 3ms/step - loss: 0.0155 - accuracy: 1.0000 - val_loss: 1.2579 - val_accuracy: 0.7895
Epoch 123/250

35/35 [==============================] - 0s 3ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 1.5557 - val_accuracy: 0.8246
Epoch 174/250
35/35 [==============================] - 0s 2ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 1.5660 - val_accuracy: 0.8187
Epoch 175/250
35/35 [==============================] - 0s 2ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 1.5627 - val_accuracy: 0.8246
Epoch 176/250
35/35 [==============================] - 0s 2ms/step - loss: 0.0108 - accuracy: 0.9920 - val_loss: 1.5701 - val_accuracy: 0.8246
Epoch 177/250
35/35 [==============================] - 0s 2ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 1.5753 - val_accuracy: 0.8246
Epoch 178/250
35/35 [==============================] - 0s 3ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 1.5832 - val_accuracy: 0.8246
Epoch 179/250
35/35 [==============================] - 0s 2ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 1.5888 - val_accuracy: 0.8363
Epoch 180/250

35/35 [==============================] - 0s 2ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 1.9048 - val_accuracy: 0.8421
Epoch 231/250
35/35 [==============================] - 0s 2ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 1.9025 - val_accuracy: 0.8421
Epoch 232/250
35/35 [==============================] - 0s 2ms/step - loss: 8.9970e-04 - accuracy: 1.0000 - val_loss: 1.9188 - val_accuracy: 0.8421
Epoch 233/250
35/35 [==============================] - 0s 3ms/step - loss: 9.9897e-04 - accuracy: 1.0000 - val_loss: 1.9252 - val_accuracy: 0.8421
Epoch 234/250
35/35 [==============================] - 0s 2ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 1.9229 - val_accuracy: 0.8421
Epoch 235/250
35/35 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 1.9325 - val_accuracy: 0.8421
Epoch 236/250
35/35 [==============================] - 0s 2ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 1.9402 - val_accuracy: 0.8421
Epoch

In [48]:
# evaluate the model1
scores = model.evaluate(X_standardized,y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

17/17 [==============================] - 0s 998us/step - loss: 0.6700 - accuracy: 0.9478
accuracy: 94.78%


In [49]:
# create model2
def create_model():
    model = Sequential()
    model.add(Dense(10, input_dim=28,activation='relu'))
    model.add(Dense(5,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    
    adam=Adam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [52]:
# Create the model2
model = KerasClassifier(build_fn = create_model,verbose = 0)
# Define the grid search parameters
batch_size = [10,20,50]
epochs = [10,50,100]
# Make a dictionary of the grid search parameters
param_grid = dict(batch_size = batch_size,epochs = epochs)
# Build and fit the GridSearchCV
grid = GridSearchCV(estimator = model,param_grid = param_grid,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............ batch_size=10, epochs=10, score=0.952, total=   1.1s
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV] ............ batch_size=10, epochs=10, score=0.817, total=   1.1s
[CV] batch_size=10, epochs=10 ........................................


...

[CV] ............ batch_size=10, epochs=50, score=0.874, total=   2.4s
[CV] batch_size=10, epochs=50 ........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   13.9s remaining:    0.0s


[CV] ............ batch_size=10, epochs=50, score=0.903, total=   2.1s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.990, total=   3.6s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.913, total=   3.8s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.893, total=   3.4s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.796, total=   3.6s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.825, total=   3.5s
[CV] batch_size=20, epochs=10 ........................................
[CV] ............ batch_size=20, epochs=10, score=0.971, total=   1.0s
[CV] batch_size=20, epochs=10 ........................................
[CV] .

[CV] ............ batch_size=50, epochs=50, score=0.913, total=   1.0s
[CV] batch_size=50, epochs=50 ........................................
[CV] ............ batch_size=50, epochs=50, score=0.854, total=   1.1s
[CV] batch_size=50, epochs=100 .......................................
[CV] ........... batch_size=50, epochs=100, score=0.990, total=   1.4s
[CV] batch_size=50, epochs=100 .......................................
[CV] ........... batch_size=50, epochs=100, score=0.885, total=   1.4s
[CV] batch_size=50, epochs=100 .......................................
[CV] ........... batch_size=50, epochs=100, score=0.922, total=   1.4s
[CV] batch_size=50, epochs=100 .......................................
[CV] ........... batch_size=50, epochs=100, score=0.893, total=   1.6s
[CV] batch_size=50, epochs=100 .......................................
[CV] ........... batch_size=50, epochs=100, score=0.874, total=   1.5s


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.2min finished


In [53]:
# Summarize the results Model 2
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.9128640651702881, using {'batch_size': 50, 'epochs': 100}
0.8373412966728211,0.07176008940192898 with: {'batch_size': 10, 'epochs': 10}
0.9012882590293885,0.05433396099349528 with: {'batch_size': 10, 'epochs': 50}
0.883681857585907,0.06847744834775633 with: {'batch_size': 10, 'epochs': 100}
0.7985997080802918,0.09284668038913571 with: {'batch_size': 20, 'epochs': 10}
0.9089805722236634,0.04190635279460872 with: {'batch_size': 20, 'epochs': 50}
0.9109036684036255,0.04040178247418451 with: {'batch_size': 20, 'epochs': 100}
0.7306571960449219,0.1328060203008201 with: {'batch_size': 50, 'epochs': 10}
0.8935959696769714,0.039090490542244505 with: {'batch_size': 50, 'epochs': 50}
0.9128640651702881,0.041977666833576 with: {'batch_size': 50, 'epochs': 100}


In [54]:
#Model 3
#Tuning of Hyperparameters:- Learning rate and Drop out rate
from keras.layers import Dropout

# Defining the model

def create_model(learning_rate,dropout_rate):
    model = Sequential()
    model.add(Dense(8,input_dim = 28,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4,input_dim = 8,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = learning_rate)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)

# Define the grid search parameters

learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]

# Make a dictionary of the grid search parameters

param_grids = dict(learning_rate = learning_rate,dropout_rate = dropout_rate)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] dropout_rate=0.0, learning_rate=0.001 ...........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  dropout_rate=0.0, learning_rate=0.001, score=1.000, total=   1.1s
[CV] dropout_rate=0.0, learning_rate=0.001 ...........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV]  dropout_rate=0.0, learning_rate=0.001, score=0.750, total=   0.9s
[CV] dropout_rate=0.0, learning_rate=0.001 ...........................


...

[CV]  dropout_rate=0.0, learning_rate=0.01, score=0.845, total=   0.8s
[CV] dropout_rate=0.0, learning_rate=0.1 .............................
[CV] . dropout_rate=0.0, learning_rate=0.1, score=0.962, total=   0.8s
[CV] dropout_rate=0.0, learning_rate=0.1 .............................
[CV] . dropout_rate=0.0, learning_rate=0.1, score=0.788, total=   1.0s
[CV] dropout_rate=0.0, learning_rate=0.1 .............................
[CV] . dropout_rate=0.0, learning_rate=0.1, score=0.786, total=   0.8s
[CV] dropout_rate=0.0, learning_rate=0.1 .............................
[CV] . dropout_rate=0.0, learning_rate=0.1, score=0.680, total=   0.9s
[CV] dropout_rate=0.0, learning_rate=0.1 .............................
[CV] . dropout_rate=0.0, learning_rate=0.1, score=0.845, total=   1.2s
[CV] dropout_rate=0.1, learning_rate=0.001 ...........................
[CV]  dropout_rate=0.1, learning_rate=0.001, score=1.000, total=   0.9s
[CV] dropout_rate=0.1, learning_rate=0.001 ...........................
[CV] 

[CV]  dropout_rate=0.1, learning_rate=0.001, score=0.680, total=   1.1s
[CV] dropout_rate=0.1, learning_rate=0.001 ...........................
[CV]  dropout_rate=0.1, learning_rate=0.001, score=0.699, total=   0.9s
[CV] dropout_rate=0.1, learning_rate=0.01 ............................
[CV]  dropout_rate=0.1, learning_rate=0.01, score=0.990, total=   1.0s
[CV] dropout_rate=0.1, learning_rate=0.01 ............................
[CV]  dropout_rate=0.1, learning_rate=0.01, score=0.750, total=   1.1s
[CV] dropout_rate=0.1, learning_rate=0.01 ............................
[CV]  dropout_rate=0.1, learning_rate=0.01, score=0.524, total=   0.9s
[CV] dropout_rate=0.1, learning_rate=0.01 ............................
[CV]  dropout_rate=0.1, learning_rate=0.01, score=0.680, total=   1.0s
[CV] dropout_rate=0.1, learning_rate=0.01 ............................
[CV]  dropout_rate=0.1, learning_rate=0.01, score=0.757, total=   1.1s
[CV] dropout_rate=0.1, learning_rate=0.1 .............................
[CV]

[CV] . dropout_rate=0.1, learning_rate=0.1, score=0.767, total=   0.8s
[CV] dropout_rate=0.1, learning_rate=0.1 .............................
[CV] . dropout_rate=0.1, learning_rate=0.1, score=0.854, total=   0.9s
[CV] dropout_rate=0.2, learning_rate=0.001 ...........................
[CV]  dropout_rate=0.2, learning_rate=0.001, score=1.000, total=   0.8s
[CV] dropout_rate=0.2, learning_rate=0.001 ...........................
[CV]  dropout_rate=0.2, learning_rate=0.001, score=0.750, total=   1.0s
[CV] dropout_rate=0.2, learning_rate=0.001 ...........................
[CV]  dropout_rate=0.2, learning_rate=0.001, score=0.524, total=   0.9s
[CV] dropout_rate=0.2, learning_rate=0.001 ...........................
[CV]  dropout_rate=0.2, learning_rate=0.001, score=0.680, total=   0.8s
[CV] dropout_rate=0.2, learning_rate=0.001 ...........................
[CV]  dropout_rate=0.2, learning_rate=0.001, score=0.699, total=   1.0s
[CV] dropout_rate=0.2, learning_rate=0.01 ............................
[

[CV]  dropout_rate=0.2, learning_rate=0.01, score=0.748, total=   1.0s
[CV] dropout_rate=0.2, learning_rate=0.01 ............................
[CV]  dropout_rate=0.2, learning_rate=0.01, score=0.699, total=   0.9s
[CV] dropout_rate=0.2, learning_rate=0.1 .............................
[CV] . dropout_rate=0.2, learning_rate=0.1, score=0.971, total=   0.9s
[CV] dropout_rate=0.2, learning_rate=0.1 .............................
[CV] . dropout_rate=0.2, learning_rate=0.1, score=0.750, total=   1.0s
[CV] dropout_rate=0.2, learning_rate=0.1 .............................
[CV] . dropout_rate=0.2, learning_rate=0.1, score=0.524, total=   0.8s
[CV] dropout_rate=0.2, learning_rate=0.1 .............................
[CV] . dropout_rate=0.2, learning_rate=0.1, score=0.709, total=   0.8s
[CV] dropout_rate=0.2, learning_rate=0.1 .............................
[CV] . dropout_rate=0.2, learning_rate=0.1, score=0.699, total=   0.8s


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   41.4s finished


In [55]:
# Summarize the results Model3
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.812135922908783, using {'dropout_rate': 0.0, 'learning_rate': 0.1}
0.7305825233459473,0.15435061319000673 with: {'dropout_rate': 0.0, 'learning_rate': 0.001}
0.7907767057418823,0.12334683292088218 with: {'dropout_rate': 0.0, 'learning_rate': 0.01}
0.812135922908783,0.09183032557822593 with: {'dropout_rate': 0.0, 'learning_rate': 0.1}
0.7305825233459473,0.15435061319000673 with: {'dropout_rate': 0.1, 'learning_rate': 0.001}
0.7403099298477173,0.15051646491703782 with: {'dropout_rate': 0.1, 'learning_rate': 0.01}
0.7675877571105957,0.13969277155200413 with: {'dropout_rate': 0.1, 'learning_rate': 0.1}
0.7305825233459473,0.15435061319000673 with: {'dropout_rate': 0.2, 'learning_rate': 0.001}
0.7578043341636658,0.11869410848998828 with: {'dropout_rate': 0.2, 'learning_rate': 0.01}
0.7306385397911072,0.14303040893944743 with: {'dropout_rate': 0.2, 'learning_rate': 0.1}


In [56]:
#Model 4
#Tuning of Hyperparameters:- Activation Function and Kernel Initializer
# Defining the model

def create_model(activation_function,init):
    model = Sequential()
    model.add(Dense(8,input_dim = 28,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(4,input_dim = 8,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 10,epochs =250)

# Define the grid search parameters
activation_function = ['softmax','relu','tanh','linear']
init = ['uniform','normal','zero']

# Make a dictionary of the grid search parameters
param_grids = dict(activation_function = activation_function,init = init)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] activation_function=softmax, init=uniform .......................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  activation_function=softmax, init=uniform, score=0.990, total=   8.3s
[CV] activation_function=softmax, init=uniform .......................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.2s remaining:    0.0s


[CV]  activation_function=softmax, init=uniform, score=0.856, total=   9.0s
[CV] activation_function=softmax, init=uniform .......................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.1min remaining:    0.0s


[CV]  activation_function=softmax, init=normal, score=0.893, total=   7.9s
[CV] activation_function=softmax, init=normal ........................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.3min remaining:    0.0s


[CV]  activation_function=softmax, init=normal, score=0.903, total=   8.1s
[CV] activation_function=softmax, init=zero ..........................
[CV]  activation_function=softmax, init=zero, score=0.990, total=   8.9s
[CV] activation_function=softmax, init=zero ..........................
[CV]  activation_function=softmax, init=zero, score=0.856, total=   7.8s
[CV] activation_function=softmax, init=zero ..........................
[CV]  activation_function=softmax, init=zero, score=0.942, total=   8.5s
[CV] activation_function=softmax, init=zero ..........................
[CV]  activation_function=softmax, init=zero, score=0.903, total=   9.4s
[CV] activation_function=softmax, init=zero ..........................
[CV]  activation_function=softmax, init=zero, score=0.883, total=   8.6s
[CV] activation_function=relu, init=uniform ..........................
[CV]  activation_function=relu, init=uniform, score=1.000, total=   7.5s
[CV] activation_function=relu, init=uniform .................

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  8.1min finished


In [57]:
# Summarize the results Model 4
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.951568341255188, using {'activation_function': 'linear', 'init': 'normal'}
0.8993278503417969,0.047971400089614105 with: {'activation_function': 'softmax', 'init': 'uniform'}
0.9187639951705933,0.04585941078833258 with: {'activation_function': 'softmax', 'init': 'normal'}
0.9148618340492248,0.0470002668216763 with: {'activation_function': 'softmax', 'init': 'zero'}
0.9283793807029724,0.0383830825548724 with: {'activation_function': 'relu', 'init': 'uniform'}
0.9264376282691955,0.04077561473635935 with: {'activation_function': 'relu', 'init': 'normal'}
0.7305825233459473,0.15435061319000673 with: {'activation_function': 'relu', 'init': 'zero'}
0.9476661682128906,0.02920387577181428 with: {'activation_function': 'tanh', 'init': 'uniform'}
0.9476848363876342,0.027161376719325194 with: {'activation_function': 'tanh', 'init': 'normal'}
0.7305825233459473,0.15435061319000673 with: {'activation_function': 'tanh', 'init': 'zero'}
0.9496265888214112,0.02837555031791039 with: {'activati

In [58]:
#Tuning of Hyperparameter :-Number of Neurons in activation layer Model 5
# Defining the model

def create_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 28,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 10,epochs = 250)

# Define the grid search parameters

neuron1 = [4,8,16]
neuron2 = [2,4,8]

# Make a dictionary of the grid search parameters

param_grids = dict(neuron1 = neuron1,neuron2 = neuron2)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(3),verbose = 10)
grid_result = grid.fit(X_standardized,y)


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ neuron1=4, neuron2=2, score=1.000, total=   6.6s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.5s remaining:    0.0s


[CV] ................ neuron1=4, neuron2=2, score=0.843, total=   7.0s
[CV] neuron1=4, neuron2=2 ............................................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   54.5s remaining:    0.0s


[CV] ................ neuron1=4, neuron2=8, score=0.919, total=   7.2s
[CV] neuron1=8, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.0min remaining:    0.0s


[CV] ................ neuron1=8, neuron2=2, score=1.000, total=   6.7s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ................ neuron1=8, neuron2=2, score=0.855, total=   6.5s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ................ neuron1=8, neuron2=2, score=0.936, total=   6.7s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ................ neuron1=8, neuron2=4, score=1.000, total=   6.2s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ................ neuron1=8, neuron2=4, score=0.820, total=   6.3s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ................ neuron1=8, neuron2=4, score=0.930, total=   6.7s
[CV] neuron1=8, neuron2=8 ............................................
[CV] ................ neuron1=8, neuron2=8, score=1.000, total=   6.8s
[CV] neuron1=8, neuron2=8 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  3.1min finished


In [59]:
# Summarize the results Model 5
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.930232564608256, using {'neuron1': 8, 'neuron2': 2}
0.9224806229273478,0.06410014378014862 with: {'neuron1': 4, 'neuron2': 2}
0.9147286812464396,0.07125883311398311 with: {'neuron1': 4, 'neuron2': 4}
0.9186046520868937,0.06645903138564027 with: {'neuron1': 4, 'neuron2': 8}
0.930232564608256,0.059480661078541275 with: {'neuron1': 8, 'neuron2': 2}
0.9166666666666666,0.07420229880214328 with: {'neuron1': 8, 'neuron2': 4}
0.9166666666666666,0.07141677743156151 with: {'neuron1': 8, 'neuron2': 8}
0.9186046520868937,0.06060657497246498 with: {'neuron1': 16, 'neuron2': 2}
0.9205426375071207,0.07204509110593683 with: {'neuron1': 16, 'neuron2': 4}
0.9205426375071207,0.06651552078052284 with: {'neuron1': 16, 'neuron2': 8}


In [60]:
# create model6
def create_model():
    model = Sequential()
    model.add(Dense(8, input_dim=28,kernel_initializer = 'normal',activation = 'linear'))
    model.add(Dropout(0.1))
    model.add(Dense(4,kernel_initializer = 'normal',activation = 'linear'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation='sigmoid'))
    
    adam=Adam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [61]:
# Create the model6 
model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 10,epochs =250)
# Build and fit the GridSearchCV
grid = GridSearchCV(estimator = model,param_grid = param_grid,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............ batch_size=10, epochs=10, score=1.000, total=   1.2s
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


[CV] ............ batch_size=10, epochs=10, score=0.865, total=   1.3s
[CV] batch_size=10, epochs=10 ........................................


...

[CV] ............ batch_size=10, epochs=50, score=0.951, total=   2.3s
[CV] batch_size=10, epochs=50 ........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   14.7s remaining:    0.0s


[CV] ............ batch_size=10, epochs=50, score=0.942, total=   2.3s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=1.000, total=   3.7s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.942, total=   3.9s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.913, total=   3.6s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.961, total=   3.6s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.922, total=   4.0s
[CV] batch_size=20, epochs=10 ........................................
[CV] ............ batch_size=20, epochs=10, score=0.990, total=   1.1s
[CV] batch_size=20, epochs=10 ........................................
[CV] .

[CV] ............ batch_size=50, epochs=50, score=0.951, total=   1.1s
[CV] batch_size=50, epochs=50 ........................................
[CV] ............ batch_size=50, epochs=50, score=0.932, total=   1.2s
[CV] batch_size=50, epochs=100 .......................................
[CV] ........... batch_size=50, epochs=100, score=1.000, total=   1.5s
[CV] batch_size=50, epochs=100 .......................................
[CV] ........... batch_size=50, epochs=100, score=0.962, total=   1.5s
[CV] batch_size=50, epochs=100 .......................................
[CV] ........... batch_size=50, epochs=100, score=0.913, total=   1.4s
[CV] batch_size=50, epochs=100 .......................................
[CV] ........... batch_size=50, epochs=100, score=0.951, total=   1.6s
[CV] batch_size=50, epochs=100 .......................................
[CV] ........... batch_size=50, epochs=100, score=0.971, total=   1.5s


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.3min finished


In [63]:
# Summarize the results Model 6
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.9592979788780213, using {'batch_size': 50, 'epochs': 100}
0.9051157593727112,0.050201542241246726 with: {'batch_size': 10, 'epochs': 10}
0.9534727334976196,0.02842611902243071 with: {'batch_size': 10, 'epochs': 50}
0.9476848483085633,0.031047757361927462 with: {'batch_size': 10, 'epochs': 100}
0.8935212850570678,0.05294107584535176 with: {'batch_size': 20, 'epochs': 10}
0.9496265888214112,0.030919047421574003 with: {'batch_size': 20, 'epochs': 50}
0.9535100936889649,0.028360213244869974 with: {'batch_size': 20, 'epochs': 100}
0.8431852102279663,0.08000544160384467 with: {'batch_size': 50, 'epochs': 10}
0.9457430958747863,0.0316559971796948 with: {'batch_size': 50, 'epochs': 50}
0.9592979788780213,0.028410807649881475 with: {'batch_size': 50, 'epochs': 100}
